## Imports and functions

In [ ]:
from pycalphad import Model, Database, variables as v
from numpy import meshgrid
from sympy import sympify, lambdify, log, latex, solve
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from symbolic_boundary import *

## Test system 1

In [ ]:
x = symbols('x')
f1 = 2*x*log(x)
f2 = (2-x)*log(2-x)+ .5

In [ ]:
b1 = boundary(f1, f2)
b2 = boundary(f2, f1)

In [ ]:
# Convert all to numeric
f1_numeric = lambdify(x, f1, 'numpy')
f2_numeric = lambdify(x, f2, 'numpy')

b1_numeric = lambdify(b1.free_symbols.pop(), b1, 'numpy')
b2_numeric = lambdify(b2.free_symbols.pop(), b2, 'numpy')

In [ ]:
# Solve the boundaries
b1_x = fsolve(b1_numeric, 0.5)
b2_x = fsolve(b2_numeric, 1.7)

In [ ]:
# Define x range while avoiding log domain issues
x_vals = np.linspace(0.01, 1.99, 400)  # Avoid x=0 and x=2 where log is undefined

# Compute y values
y1_vals = f1_numeric(x_vals)
y2_vals = f2_numeric(x_vals)

# Plot the functions
plt.figure(figsize=(8, 6))
plt.plot(x_vals, y1_vals, label=rf'$f_1(x) = {latex(f1)}$', color='blue')
plt.plot(x_vals, y2_vals, label=rf'$f_2(x) = {latex(f2)}$', color='red')

# Plot the boundaries
plt.scatter(b1_x, f1_numeric(b1_x), label='Boundary Point 1', color='blue')
plt.scatter(b2_x, f2_numeric(b2_x), label='Boundary Point 2', color='red')

# Formatting the plot
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('Plot of $f_1(x)$ and $f_2(x)$')
plt.xlim(0, 2)
plt.ylim(-1, 1)
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

## Test system 2

In [ ]:
x = symbols('x')
f1 = x**(-10) + log(x)
f2 = x**(-1) + x*log(x) + x + x**2 - 2

In [ ]:
solve(x - log(x))

In [ ]:
solve(diff(x**(-1) + x*log(x) + x + x**2 - 2, x), x)

In [ ]:
# b1 = boundary(f1, f2)
# b2 = boundary(f2, f1)

In [ ]:
# Convert all to numeric
f1_numeric = lambdify(x, f1, 'numpy')
f2_numeric = lambdify(x, f2, 'numpy')

b1_numeric = lambdify(b1.free_symbols.pop(), b1, 'numpy')
b2_numeric = lambdify(b2.free_symbols.pop(), b2, 'numpy')

In [ ]:
# Solve the boundaries
b1_x = fsolve(b1_numeric, 0.5)
b2_x = fsolve(b2_numeric, 1.7)

In [ ]:
# Define x range while avoiding log domain issues
x_vals = np.linspace(0.01, 1.99, 400)  # Avoid x=0 and x=2 where log is undefined

# Compute y values
y1_vals = f1_numeric(x_vals)
y2_vals = f2_numeric(x_vals)

# Plot the functions
plt.figure(figsize=(8, 6))
plt.plot(x_vals, y1_vals, label=rf'$f_1(x) = {latex(f1)}$', color='blue')
plt.plot(x_vals, y2_vals, label=rf'$f_2(x) = {latex(f2)}$', color='red')

# Plot the boundaries
plt.scatter(b1_x, f1_numeric(b1_x), label='Boundary Point 1', color='blue')
plt.scatter(b2_x, f2_numeric(b2_x), label='Boundary Point 2', color='red')

# Formatting the plot
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('Plot of $f_1(x)$ and $f_2(x)$')
plt.xlim(0, 2)
plt.ylim(0, 2)
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

## We will start with the a binary system just so that we can visualize it

In [ ]:
db = Database("../TDDatabaseFiles_temp/pbsn.tdb")
phases = list(db.phases.keys())
print(phases) 

In [ ]:
phase = 'FCC_A1'
comps = [list(sublattice)[0].name for sublattice in db.phases[phase].constituents]
model = Model(db, comps, phase)

sp_energy_expr = sympify(model.HM)  # Get the Enthaply expression in sympy
display(sp_energy_expr)

In [ ]:
# Get the chemical variables only
chem_symbols = model.HM.free_symbols
chem_symbols.remove(v.T)
print(chem_symbols)

In [ ]:
# Variable dictionary here
x, T, v = symbols('x T v')
symbol_dict = {
    'AL': x,
    'FE': 1 - x,
    'T': T,
    'VA': v
    }

for chem_symbol in chem_symbols:
    name = chem_symbol.species.name
    sp_energy_expr = sp_energy_expr.subs(chem_symbol, symbol_dict[name])

display(sp_energy_expr)

In [ ]:
energy_numeric = lambdify([x, T], sp_energy_expr, 'numpy')

# Check
print(sp_energy_expr.subs(v.T, 1000).subs(x, 0.5).evalf())
print(energy_numeric(0.5, 1000))

In [ ]:
# This is just to visualize all real energy functions
X = np.linspace(0, 1, 100)
T = np.linspace(300, 2000, 100)

X, T = meshgrid(X, T)
G = energy_numeric(X, T)

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Surface(
        x=X, y=T, z=G,
        name='G'
    )
)

fig.update_layout(
    title="Energy function", 
    scene=dict(
        aspectmode='cube',
        xaxis_title='Al',
        yaxis_title='T',
        zaxis_title='G'
    ),
    width=800,
    height=700,
)

fig.show()

In [ ]:
# def build_polynomial_basis(x, y, n):
#     """
#     Returns a list of Sympy monomials
#     """
#     basis = [1]
#     for i in range(1, n+1):
#         basis.append(x**i)
#     for i in range(1, n+1):
#         basis.append(y**i)

#     return basis

# def double_integral(sym_expression):
#     """
#     Numerically compute the integral of sym_expression(x,y) 
#     """
#     # Convert the sympy expression to a numeric Python function f_num(x,y).
#     f_num = sp.lambdify((x, y), sym_expression, 'numpy')
    
#     def integrand(yval, xval):
#         return f_num(xval, yval)
    
#     val, err = spi.dblquad(
#         integrand,
#         0,   # x-lower
#         1,   # x-upper
#         lambda xval: 0,  # y-lower
#         lambda xval: 1 - xval  # y-upper; ternaty system
#     )
#     return val

# def fit_ternary_polynomial_galerkin(f_expr, n):
#     """
#     Fits a 2D polynomial p(x,y) = sum_{i=0..n, j=0..n} alpha_{i,j} * x^i y^j
#     to the given Sympy expression f_expr(x,y), via
#     least-squares on [0,1]^2. Returns the symbolic polynomial approximation.
#     """
#     # Build basis with cross terms
#     basis = build_polynomial_basis(x, y, n)
#     n_basis = len(basis)  # = (n+1)^2

#     # Construct the Gram matrix M and vector b
#     M = sp.zeros(n_basis, n_basis)
#     b_vec = sp.zeros(n_basis, 1)

#     for i in range(n_basis):
#         for j in range(n_basis):
#             M[i, j] = double_integral(basis[i]*basis[j])
#         b_vec[i, 0] = double_integral(f_expr*basis[i])

#     # Solve the linear system M alpha = b
#     alpha = M.LUsolve(b_vec)

#     # Build the final polynomial
#     p_approx = sum(alpha[i,0] * basis[i] for i in range(n_basis))
#     p_approx_simple = sp.simplify(p_approx)
    
#     return p_approx_simple

In [ ]:
# def compute_boundary_points(boundary, mesh_density):
#     '''
#     Compute the boundary points for the given boundary equation
#     '''
#     # Generate the 1D mesh
#     space = np.linspace(0, 1, mesh_density)
#     x, y = symbols('x y')

#     # Solve boundary=0 for y (treating x as the independent variable)
#     solutions_y = sp.solve(sp.Eq(boundary, 0), y)
#     # Solve boundary=0 for x (treating y as the independent variable)
#     solutions_x = sp.solve(sp.Eq(boundary, 0), x)
    
#     # Convert each Sympy solution to a NumPy-callable function
#     sol_y_lambdas = [sp.lambdify(x, sol, 'numpy') for sol in solutions_y]
#     sol_x_lambdas = [sp.lambdify(y, sol, 'numpy') for sol in solutions_x]

#     if sol_y_lambdas == [] and sol_x_lambdas == []:
#         return np.array([]), np.array([])

#     # We'll accumulate all x-values and y-values in lists and concatenate once.
#     x_vals = []
#     y_vals = []
    
#     # Evaluate y = f(x) for each solution
#     for f_y in sol_y_lambdas:
#         x_vals.append(space)
#         y_vals.append(f_y(space))
    
#     # Evaluate x = f(y) for each solution
#     for f_x in sol_x_lambdas:
#         x_vals.append(f_x(space))
#         y_vals.append(space)
    
#     # Combine into final arrays
#     X = np.concatenate(x_vals)
#     Y = np.concatenate(y_vals)
    
#     # Apply the mask: only keep points where X + Y <= 1
#     # Replace invalid points with np.nan (or you could filter them out)
#     mask = ((X + Y) <= 1) & (X >= 0) & (Y >= 0)
#     X = X[mask]
#     Y = Y[mask]
    
#     return X, Y

## Real system

In [ ]:
# # Teperature
# temperature = 200

# # Variable dictionary here
# x, y = symbols('x y')
# symbol_dict = {
#     'CU': x,
#     'CU2': 2*x,
#     'AL': y,
#     'Y': 1-x-y,
#     'VA': 1}

# # Polynomial order
# n = 2

In [ ]:
# db = Database("../TDDatabaseFiles_temp/ALCUY-2011.TDB")
# # phases = list(db.phases.keys())
# phases = ['AL42CU68Y10']    # Test phases
# # phases = ['CU6Y']
# print(phases)

In [ ]:
# # Create a dictionary to store the polynomial expressions
# phase_energy_expression_dict = dict()

# # First we need to go through all phases and fit polynomials to their energy functions
# for phase in phases:
#     comps = [list(sublattice)[0].name for sublattice in db.phases[phase].constituents]
#     model = Model(db, comps, phase)

#     sp_energy_expr = sp.sympify(model.GM)  # Get the Gibbs energy expression in sympy

#     # Get the symbols from the non-converted expression
#     chem_symbols = model.HM.free_symbols
#     # chem_symbols.remove(v.T)

#     display(sp_energy_expr)


#     # # Evaluate the function at some temperature
#     # sp_energy_expr = sp_energy_expr.subs(v.T, temperature)
#     # for chem_symbol in chem_symbols:
#     #     name = chem_symbol.species.name
#     #     sp_energy_expr = sp_energy_expr.subs(chem_symbol, symbol_dict[name])

#     # # Fit a polynomial to the energy function
#     # print('Fitting polynomial to', phase)
#     # poly_energy_expr = fit_ternary_polynomial_galerkin(sp_energy_expr, n)

#     # # Round off the polynomials
#     # poly_energy_expr = poly_energy_expr.replace(lambda term: term.is_Number, lambda term: int(round(term, 0)))

#     # # Add the polynomial to the dictionary
#     # phase_energy_expression_dict[phase] = [poly_energy_expr, sp.lambdify((x, y), sp_energy_expr, 'numpy')]    # We store both the polynomial and the original expression as a vectorized function

In [ ]:
# # This is just to visualize all real energy functions

# fig = go.Figure()
# X, Y = np.meshgrid(np.linspace(0, 1, 100), np.linspace(0, 1, 100))
# mask = X + Y <= 1
# X, Y = np.where(mask, X, np.nan), np.where(mask, Y, np.nan)

# for phase in phase_energy_expression_dict:
#     energy_func = phase_energy_expression_dict[phase][1]
#     Z = energy_func(X, Y)
#     fig.add_trace(go.Surface(z=Z, x=X, y=Y, name=phase, showscale=False))

# fig.update_layout(
#     title="Energy functions", 
#     scene=dict(
#         aspectmode='cube',
#         xaxis_title='Cu',
#         yaxis_title='Al',
#         zaxis_title='G'
#     ),
#     width=800,
#     height=700,
# )

# fig.show()

In [ ]:
# phase_pairs = list(combinations(phase_energy_expression_dict, 2))
# phase_pairs_dict = dict()

# for pair in phase_pairs:
#     # Compute the phase boundaries
#     boundary1 = hpboundry_f2_to_f1(phase_energy_expression_dict[pair[0]][0], phase_energy_expression_dict[pair[1]][0])
#     boundary2 = hpboundry_f2_to_f1(phase_energy_expression_dict[pair[1]][0], phase_energy_expression_dict[pair[0]][0])

#     phase_pairs_dict[pair] = boundary1
#     # Reverse the phase pairs and add the other boundary
#     phase_pairs_dict[(pair[1], pair[0])] = boundary2

In [ ]:
# # Record the phase that a point is lying on
# boundary_points_lists = dict()

# # Got through all the phase pairs and compute the boundary points
# mesh_density = 100
# for pair in phase_pairs_dict:
#     # Get the boundary points in X and Y
#     Xline, Yline = compute_boundary_points(phase_pairs_dict[pair], mesh_density)

#     # The polynomial isnt the best fit to the energy so we will compute actual energies off of the true function
#     energy_lambda = sp.lambdify((x, y), phase_energy_expression_dict[pair[0]][0], 'numpy')
#     Energies = energy_lambda(Xline, Yline)

#     # Aggregate all the points
#     data = np.column_stack((Xline, Yline, Energies))

#     # Initialize a list for this phase if it doesn't exist yet
#     if phase not in boundary_points_lists:
#         boundary_points_lists[pair[0]] = []

#     # Append the new data to the list
#     boundary_points_lists[pair[0]].append(data)

# boundary_points = {}
# for phase, data_list in boundary_points_lists.items():
#     boundary_points[phase] = np.vstack(data_list)

In [ ]:
# # Create a composition space to plot the phases
# X = np.linspace(0, 1, 100) 
# Y = np.linspace(0, 1, 100)

# # Create meshgrid
# X, Y = np.meshgrid(X, Y)
# mask = (X + Y) <= 1

# # Create a triangular mask for the ternary system
# X, Y = np.where(mask, X, np.nan), np.where(mask, Y, np.nan)

In [ ]:
# # Plot with Plotly
# fig = go.Figure()

# for phase in phases:

#     # Add the energy function
#     fig.add_trace(
#         go.Surface(
#             x=X, y=Y, z=sp.lambdify((x, y), phase_energy_expression_dict[phase][0], 'numpy')(X, Y),
#             colorscale='RdBu',
#             opacity=0.7,
#             name=phase,
#             showscale=False
#         )
#     )

#     # Add the boundary points
#     fig.add_trace(
#         go.Scatter3d(
#             x=boundary_points[phase][:,0], y=boundary_points[phase][:,1], z=boundary_points[phase][:,2],
#             mode='markers',
#             marker=dict(size=5),
#             name=f'{phase} boundary points'
#         )
#     )

# fig.update_layout(
#     title="Energy functions and boundaries", 
#     scene=dict(
#         aspectmode='cube',
#         xaxis_title='Cu',
#         yaxis_title='Al',
#         zaxis_title='G'
#     ),
#     width=800,
#     height=700
# )

# fig.show()